In [21]:
import zipfile
import pathlib
import string
from collections import Counter
from typing import List

import networkx as nx

In [22]:
%load_ext autoreload

%autoreload 2

from utils import read_text, preprocess_text
from language_model import count_ngram, NGramStat
from encoder import beam_search, build_graph, decode_ngrams, encode_text, decode_text

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [23]:
path_to_zip = pathlib.Path("data", "corpora.zip")

In [24]:
texts = read_text(path_to_zip, ignore_files=["WarAndPeaceEng.txt"])

In [25]:
n_gram = 1

In [26]:
source_freq = Counter()
for name in texts:
    if name == "AnnaKarenina.txt":
        big_text = texts[name]
    else:
        source_freq.update(count_ngram(texts[name], n_gram=n_gram))

In [27]:
unigram_stat = NGramStat(source_freq)

In [28]:
vocab = set(unigram_stat._ngram_proba.keys())

# Частотное декодирование на основе частот отдельных симовлов

In [29]:
test_text = "Вронский слушал с удовольствием этот веселый лепет хорошенькой женщины, поддакивая ей, давал полушутливые советы и вообще тотчас же принял свой привычный тон обращения с этого рода женщинами. В его петербургском мире все люди разделялись на два совершенно противоположные сорта. Один низший сорт: пошлые, глупые и, главное, смешные люди, которые веруют в то, что одному мужу надо жить с одною женой, с которою он обвенчан, что девушке надо быть невинною, женщине стыдливою, мужчине мужественным, воздержным и твердым, что надо воспитывать детей, зарабатывать свой хлеб, платить долги, — и разные тому подобные глупости. Это был сорт людей старомодных и смешных. Но был другой сорт людей, настоящих, к которому они все принадлежали, в котором надо быть, главное, элегантным, красивым, великодушным, смелым, веселым, отдаваться всякой страсти не краснея и над всем остальным смеяться."

In [30]:
encoded_text, original_processed = encode_text(test_text, vocab, n_gram=1)

Encoded text length: 848


In [31]:
encoded_text

'🚿🍷🏪🏉🎱🚗🗽🎈🎓🎱🚏📅🎍🕢🚏🎓🎱🎓📅💢🏪🚿🏪🚏🚖🎱🙊🚿🗽🙇🚾🎓🍖🙊🏪🙊🎓🚿🙇🎱🙇🚏🏣🎈🎓🚏🙇👧🙇🙊🎓💴🏪🍷🏪🎍🙇🏉🚖🚗🏪🎈🎓🚡🙇🏉🍑🗽🏉🏣🎓👧🏪💢💢🕢🚗🗽🚿🕢🐹🎓🙇🎈🎓💢🕢🚿🕢🚏🎓👧🏪🚏📅🎍📅🙊🚏🗽🚿🏣🙇🎓🎱🏪🚿🙇🙊🏣🎓🗽🎓🚿🏪🏪🚞🍑🙇🎓🙊🏪🙊💧🕢🎱🎓🚡🙇🎓👧🍷🗽🏉🐹🚏🎓🎱🚿🏪🎈🎓👧🍷🗽🚿🏣💧🏉🏣🎈🎓🙊🏪🏉🎓🏪🚞🍷🕢🍑🙇🏉🗽🐹🎓🎱🎓🍖🙊🏪🏤🏪🎓🍷🏪💢🕢🎓🚡🙇🏉🍑🗽🏉🕢🚾🗽🎓🚿🎓🙇🏤🏪🎓👧🙇🙊🙇🍷🚞📅🍷🏤🎱🚗🏪🚾🎓🚾🗽🍷🙇🎓🚿🎱🙇🎓🚏🚑💢🗽🎓🍷🕢📇💢🙇🚏🐹🚏🗽🎱🚖🎓🏉🕢🎓💢🚿🕢🎓🎱🏪🚿🙇🍷🎍🙇🏉🏉🏪🎓👧🍷🏪🙊🗽🚿🏪👧🏪🚏🏪🚡🏉🏣🙇🎓🎱🏪🍷🙊🕢🎓🏪💢🗽🏉🎓🏉🗽📇🎍🗽🎈🎓🎱🏪🍷🙊🎓👧🏪🎍🚏🏣🙇🎓🏤🚏📅👧🏣🙇🎓🗽🎓🏤🚏🕢🚿🏉🏪🙇🎓🎱🚾🙇🎍🏉🏣🙇🎓🚏🚑💢🗽🎓🚗🏪🙊🏪🍷🏣🙇🎓🚿🙇🍷📅🚑🙊🎓🚿🎓🙊🏪🎓💧🙊🏪🎓🏪💢🏉🏪🚾📅🎓🚾📅🚡📅🎓🏉🕢💢🏪🎓🚡🗽🙊🚖🎓🎱🎓🏪💢🏉🏪🚑🎓🚡🙇🏉🏪🎈🎓🎱🎓🚗🏪🙊🏪🍷🏪🚑🎓🏪🏉🎓🏪🚞🚿🙇🏉💧🕢🏉🎓💧🙊🏪🎓💢🙇🚿📅🎍🚗🙇🎓🏉🕢💢🏪🎓🚞🏣🙊🚖🎓🏉🙇🚿🗽🏉🏉🏪🚑🎓🚡🙇🏉🍑🗽🏉🙇🎓🎱🙊🏣💢🚏🗽🚿🏪🚑🎓🚾📅🚡💧🗽🏉🙇🎓🚾📅🚡🙇🎱🙊🚿🙇🏉🏉🏣🚾🎓🚿🏪📇💢🙇🍷🚡🏉🏣🚾🎓🗽🎓🙊🚿🙇🍷💢🏣🚾🎓💧🙊🏪🎓🏉🕢💢🏪🎓🚿🏪🎱👧🗽🙊🏣🚿🕢🙊🚖🎓💢🙇🙊🙇🎈🎓📇🕢🍷🕢🚞🕢🙊🏣🚿🕢🙊🚖🎓🎱🚿🏪🎈🎓💴🚏🙇🚞🎓👧🚏🕢🙊🗽🙊🚖🎓💢🏪🚏🏤🗽🎓🗽🎓🍷🕢📇🏉🏣🙇🎓🙊🏪🚾📅🎓👧🏪💢🏪🚞🏉🏣🙇🎓🏤🚏📅👧🏪🎱🙊🗽🎓🍖🙊🏪🎓🚞🏣🚏🎓🎱🏪🍷🙊🎓🚏🚑💢🙇🎈🎓🎱🙊🕢🍷🏪🚾🏪💢🏉🏣💴🎓🗽🎓🎱🚾🙇🎍🏉🏣💴🎓🏉🏪🎓🚞🏣🚏🎓💢🍷📅🏤🏪🎈🎓🎱🏪🍷🙊🎓🚏🚑💢🙇🎈🎓🏉🕢🎱🙊🏪🐹🍑🗽💴🎓🚗🎓🚗🏪🙊🏪🍷🏪🚾📅🎓🏪🏉🗽🎓🚿🎱🙇🎓👧🍷🗽🏉🕢💢🚏🙇🚡🕢🚏🗽🎓🚿🎓🚗🏪🙊🏪🍷🏪🚾🎓🏉🕢💢🏪🎓🚞🏣🙊🚖🎓🏤🚏🕢🚿🏉🏪🙇🎓🍖🚏🙇🏤🕢🏉🙊🏉🏣🚾🎓🚗🍷🕢🎱🗽🚿🏣🚾🎓🚿🙇🚏🗽🚗🏪💢📅🎍🏉🏣🚾🎓🎱🚾🙇🚏🏣🚾🎓🚿🙇🎱🙇🚏🏣🚾🎓🏪🙊💢🕢🚿🕢🙊🚖🎱🐹🎓🚿🎱🐹🚗🏪🎈🎓🎱🙊🍷🕢🎱🙊🗽🎓🏉🙇🎓🚗🍷🕢🎱🏉🙇🐹🎓🗽🎓🏉🕢💢🎓🚿🎱🙇🚾🎓🏪🎱🙊🕢🚏🚖🏉🏣🚾🎓🎱🚾🙇🐹🙊🚖🎱🐹'

In [32]:
for serach_res in decode_text(encoded_text, unigram_stat, n_gram, beam_search_width=10):
    print(serach_res.phrase)

Beam search: 17004it [00:30, 561.56it/s]  
Beam search:   0%|          | 0/847 [00:00<?, ?it/s]Start form: (0, 'с')
сроенятг нвублв н укосовьнистам эиои санавдг вапаи юоробаеьяог ыаехтед покклятслч аг клслв повубуивтсда носаид т соозха иоишлн ыа пртечв нсог пртсдшедг иое озрлхаетч н эиойо рокл ыаехтелмт с айо паиарзурйняом мтра сна вжкт рлцкавчвтнь ел ксл носарбаеео проитсоповоыеда норил окте етцбтг нори побвда йвупда т йвлсеоа нмабеда вжкт яоиорда саружи с ио шио океому муыу елко ытиь н океож ыаеог н яоиорож ое озсаешле шио касубяа елко здиь еастееож ыаехтеа нидквтсож муыштеа муыанисаеедм соцкарыедм т исаркдм шио елко сонптидслиь каиаг цлрлзлидслиь нсог юваз пвлитиь ковйт т рлцеда иому покозеда йвупонит эио здв нори вжкаг нилромокедю т нмабедю ео здв круйог нори вжкаг елниочхтю я яоиорому оет сна пртелкваылвт с яоиором елко здиь йвлсеоа эвайлеиедм ярлнтсдм савтяокубедм нмавдм санавдм оиклслиьнч снчяог нирлнит еа ярлнеач т елк снам онилвьедм нмачиьнч


In [33]:
original_processed

'вронский слушал с удовольствием этот веселый лепет хорошенькой женщины поддакивая ей давал полушутливые советы и вообще тотчас же принял свой привычный тон обращения с этого рода женщинами в его петербургском мире все люди разделялись на два совершенно противоположные сорта один низший сорт пошлые глупые и главное смешные люди которые веруют в то что одному мужу надо жить с одною женой с которою он обвенчан что девушке надо быть невинною женщине стыдливою мужчине мужественным воздержным и твердым что надо воспитывать детей зарабатывать свой хлеб платить долги и разные тому подобные глупости это был сорт людей старомодных и смешных но был другой сорт людей настоящих к которому они все принадлежали в котором надо быть главное элегантным красивым великодушным смелым веселым отдаваться всякой страсти не краснея и над всем остальным смеяться'

# Частотное декодирование на основе частот биграмм

In [34]:
n_gram = 2
source_freq = Counter()
for name in texts:
    if name == "AnnaKarenina.txt":
        big_text = texts[name]
    else:
        source_freq.update(count_ngram(texts[name], n_gram=n_gram))

In [35]:
bigram_stat = NGramStat(source_freq)

In [36]:
encoded_text, original_processed = encode_text(test_text, vocab, n_gram=n_gram)

Encoded text length: 848


In [37]:
encoded_text

'🚿🍷🏪🏉🎱🚗🗽🎈🎓🎱🚏📅🎍🕢🚏🎓🎱🎓📅💢🏪🚿🏪🚏🚖🎱🙊🚿🗽🙇🚾🎓🍖🙊🏪🙊🎓🚿🙇🎱🙇🚏🏣🎈🎓🚏🙇👧🙇🙊🎓💴🏪🍷🏪🎍🙇🏉🚖🚗🏪🎈🎓🚡🙇🏉🍑🗽🏉🏣🎓👧🏪💢💢🕢🚗🗽🚿🕢🐹🎓🙇🎈🎓💢🕢🚿🕢🚏🎓👧🏪🚏📅🎍📅🙊🚏🗽🚿🏣🙇🎓🎱🏪🚿🙇🙊🏣🎓🗽🎓🚿🏪🏪🚞🍑🙇🎓🙊🏪🙊💧🕢🎱🎓🚡🙇🎓👧🍷🗽🏉🐹🚏🎓🎱🚿🏪🎈🎓👧🍷🗽🚿🏣💧🏉🏣🎈🎓🙊🏪🏉🎓🏪🚞🍷🕢🍑🙇🏉🗽🐹🎓🎱🎓🍖🙊🏪🏤🏪🎓🍷🏪💢🕢🎓🚡🙇🏉🍑🗽🏉🕢🚾🗽🎓🚿🎓🙇🏤🏪🎓👧🙇🙊🙇🍷🚞📅🍷🏤🎱🚗🏪🚾🎓🚾🗽🍷🙇🎓🚿🎱🙇🎓🚏🚑💢🗽🎓🍷🕢📇💢🙇🚏🐹🚏🗽🎱🚖🎓🏉🕢🎓💢🚿🕢🎓🎱🏪🚿🙇🍷🎍🙇🏉🏉🏪🎓👧🍷🏪🙊🗽🚿🏪👧🏪🚏🏪🚡🏉🏣🙇🎓🎱🏪🍷🙊🕢🎓🏪💢🗽🏉🎓🏉🗽📇🎍🗽🎈🎓🎱🏪🍷🙊🎓👧🏪🎍🚏🏣🙇🎓🏤🚏📅👧🏣🙇🎓🗽🎓🏤🚏🕢🚿🏉🏪🙇🎓🎱🚾🙇🎍🏉🏣🙇🎓🚏🚑💢🗽🎓🚗🏪🙊🏪🍷🏣🙇🎓🚿🙇🍷📅🚑🙊🎓🚿🎓🙊🏪🎓💧🙊🏪🎓🏪💢🏉🏪🚾📅🎓🚾📅🚡📅🎓🏉🕢💢🏪🎓🚡🗽🙊🚖🎓🎱🎓🏪💢🏉🏪🚑🎓🚡🙇🏉🏪🎈🎓🎱🎓🚗🏪🙊🏪🍷🏪🚑🎓🏪🏉🎓🏪🚞🚿🙇🏉💧🕢🏉🎓💧🙊🏪🎓💢🙇🚿📅🎍🚗🙇🎓🏉🕢💢🏪🎓🚞🏣🙊🚖🎓🏉🙇🚿🗽🏉🏉🏪🚑🎓🚡🙇🏉🍑🗽🏉🙇🎓🎱🙊🏣💢🚏🗽🚿🏪🚑🎓🚾📅🚡💧🗽🏉🙇🎓🚾📅🚡🙇🎱🙊🚿🙇🏉🏉🏣🚾🎓🚿🏪📇💢🙇🍷🚡🏉🏣🚾🎓🗽🎓🙊🚿🙇🍷💢🏣🚾🎓💧🙊🏪🎓🏉🕢💢🏪🎓🚿🏪🎱👧🗽🙊🏣🚿🕢🙊🚖🎓💢🙇🙊🙇🎈🎓📇🕢🍷🕢🚞🕢🙊🏣🚿🕢🙊🚖🎓🎱🚿🏪🎈🎓💴🚏🙇🚞🎓👧🚏🕢🙊🗽🙊🚖🎓💢🏪🚏🏤🗽🎓🗽🎓🍷🕢📇🏉🏣🙇🎓🙊🏪🚾📅🎓👧🏪💢🏪🚞🏉🏣🙇🎓🏤🚏📅👧🏪🎱🙊🗽🎓🍖🙊🏪🎓🚞🏣🚏🎓🎱🏪🍷🙊🎓🚏🚑💢🙇🎈🎓🎱🙊🕢🍷🏪🚾🏪💢🏉🏣💴🎓🗽🎓🎱🚾🙇🎍🏉🏣💴🎓🏉🏪🎓🚞🏣🚏🎓💢🍷📅🏤🏪🎈🎓🎱🏪🍷🙊🎓🚏🚑💢🙇🎈🎓🏉🕢🎱🙊🏪🐹🍑🗽💴🎓🚗🎓🚗🏪🙊🏪🍷🏪🚾📅🎓🏪🏉🗽🎓🚿🎱🙇🎓👧🍷🗽🏉🕢💢🚏🙇🚡🕢🚏🗽🎓🚿🎓🚗🏪🙊🏪🍷🏪🚾🎓🏉🕢💢🏪🎓🚞🏣🙊🚖🎓🏤🚏🕢🚿🏉🏪🙇🎓🍖🚏🙇🏤🕢🏉🙊🏉🏣🚾🎓🚗🍷🕢🎱🗽🚿🏣🚾🎓🚿🙇🚏🗽🚗🏪💢📅🎍🏉🏣🚾🎓🎱🚾🙇🚏🏣🚾🎓🚿🙇🎱🙇🚏🏣🚾🎓🏪🙊💢🕢🚿🕢🙊🚖🎱🐹🎓🚿🎱🐹🚗🏪🎈🎓🎱🙊🍷🕢🎱🙊🗽🎓🏉🙇🎓🚗🍷🕢🎱🏉🙇🐹🎓🗽🎓🏉🕢💢🎓🚿🎱🙇🚾🎓🏪🎱🙊🕢🚏🚖🏉🏣🚾🎓🎱🚾🙇🐹🙊🚖🎱🐹'

In [38]:
for serach_res in decode_text(encoded_text, bigram_stat, n_gram, beam_search_width=20):
    print()
    print(serach_res.log_likelihood, serach_res.phrase)
    print()

Beam search: 848it [00:09, 85.53it/s]               
Beam search: 5836it [00:00, 58341.42it/s]         Start form: (0, 'у')
Beam search: 15394it [00:00, 20356.18it/s]
-825.2856673436542 уюпеётетбоъяф цуо шлимьсчеакакгоэтмуенут о и велциалф  вдогрсяблдой коанлеалылнощеан сл  санься инмоятчтмибыфешкенвальчтм умжнелчтковоя бочаё ковочукаё м анасу хоосщепеэт треы алелдой неоленситоенцужддрф ётс тоспчт пчтдикиузыеа  омору нтеантемибыовдолиены зьчаъюсев у тои отелеронзашаетбои маъюыму т ъяывчтцефиа личтвиа качтдикиеммуф у енждатматом енм елери рашиютязваревожабоелеринблдосябоеммуф  н санасбыотноннм есгля  кязвареи жа н иролих ъеотрочтнаодлочах шикнрочтшиъенабыосщиеназа ивкагоцеакжд бгопоренестенегдотьножа сцуднпуые нниеннию олся двебуатниотю стшецецее цау омс раъюстску т ъяъенацеэтреи цумицобиже нбогах  керня кболапрняязи и цемнютсябои мадие ё нена сй ь мея м ы шийстре таановнеалыш сценея м ы гонест идую фиа личтфресны нщиеме шлятгобымой елпрщибои мегобыжамегомуалножавые выя ё нае нацен еме 

In [39]:
original_processed

'вронский слушал с удовольствием этот веселый лепет хорошенькой женщины поддакивая ей давал полушутливые советы и вообще тотчас же принял свой привычный тон обращения с этого рода женщинами в его петербургском мире все люди разделялись на два совершенно противоположные сорта один низший сорт пошлые глупые и главное смешные люди которые веруют в то что одному мужу надо жить с одною женой с которою он обвенчан что девушке надо быть невинною женщине стыдливою мужчине мужественным воздержным и твердым что надо воспитывать детей зарабатывать свой хлеб платить долги и разные тому подобные глупости это был сорт людей старомодных и смешных но был другой сорт людей настоящих к которому они все принадлежали в котором надо быть главное элегантным красивым великодушным смелым веселым отдаваться всякой страсти не краснея и над всем остальным смеяться'